In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, GRU, Bidirectional
import numpy as np

2024-07-22 10:17:50.758221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Database
https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs

# Veamos los datos


In [2]:
import pandas as pd

dataset = pd.read_csv("googl.us.txt", delimiter=",")


In [3]:
googl_op = dataset['Open']
googl_cl = dataset['Close']

In [4]:
import numpy as np
xx = googl_op.values[np.arange(0,2000,1)]
yy = googl_cl.values[np.arange(0,2000,1)]
                     
X_train = np.reshape(xx,(200,10,1))
y_train = yy[np.arange(9,2000,10)]
                     
xx = googl_op.values[np.arange(2000,3000,1)]
yy = googl_cl.values[np.arange(2000,3000,1)]
X_test = np.reshape(xx,(100,10,1))
y_test = yy[np.arange(9,1000,10)]

# MODELO 1

In [5]:
model = Sequential()

model.add(Bidirectional(GRU(20), input_shape=(10,1), merge_mode="ave"))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mae')


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 20)               2760      
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 2,781
Trainable params: 2,781
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
7/7 [==============================] - 4s 66ms/step - loss: 228.6119
Epoch 2/10
7/7 [==============================] - 0s 52ms/step - loss: 228.3296
Epoch 3/10
7/7 [==============================] - 0s 35ms/step - loss: 228.2176
Epoch 4/10
7/7 [==============================] - 0s 30ms/step - loss: 228.0450
Epoch 5/10
7/7 [==============================] - 0s 28ms/step - loss: 227.6208
Epoch 6/10
7/7 [==============================] - 0s 27ms/step - loss: 227.3495
Epoch 7/10
7/7 [==============================] - 0s 37ms/step - loss: 227.0814
Epoch 8/10
7/7 [==============================] - 0s 31ms/step - loss: 226.7949
Epoch 9/10
7/7 [==============================] - 0s 40ms/step - loss: 226.5935
Epoch 10/10
7/7 [==============================] - 0s 37ms/step - loss: 226.4617


In [8]:

# EVALUACION
score = model.evaluate(X_test, y_test, batch_size=128)

score

1/1 [==============================] - 1s 676ms/step - loss: 548.2633


548.2633056640625

# MODELO 2

Bidireccional asimétrica

In [9]:
model = Sequential()

forward_layer = GRU(20)
backward_layer = LSTM(20, activation='relu',
                       go_backwards=True)

model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                         input_shape=(10,1)))
 
model.add(Dense(1))

model.compile(optimizer='adam',loss='mae')

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 40)               3140      
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 41        
                                                                 
Total params: 3,181
Trainable params: 3,181
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
7/7 [==============================] - 3s 21ms/step - loss: 259.6206
Epoch 2/10
7/7 [==============================] - 0s 33ms/step - loss: 254.1486
Epoch 3/10
7/7 [==============================] - 0s 26ms/step - loss: 249.8613
Epoch 4/10
7/7 [==============================] - 0s 25ms/step - loss: 239.1718
Epoch 5/10
7/7 [==============================] - 0s 25ms/step - loss: 232.9234
Epoch 6/10
7/7 [==============================] - 0s 30ms/step - loss: 230.5293
Epoch 7/10
7/7 [==============================] - 0s 30ms/step - loss: 228.1489
Epoch 8/10
7/7 [==============================] - 0s 25ms/step - loss: 217.4758
Epoch 9/10
7/7 [==============================] - 0s 25ms/step - loss: 137.1872
Epoch 10/10
7/7 [==============================] - 0s 26ms/step - loss: 45.7811


In [12]:

# EVALUACION
score = model.evaluate(X_test, y_test, batch_size=128)

score

1/1 [==============================] - 1s 651ms/step - loss: 14.9382


14.938180923461914

# MODELO 3

Bidireccional asimétrica con secuencias

In [13]:
model = Sequential()

forward_layer = GRU(20, return_sequences=True)
backward_layer = GRU(20, activation='relu', return_sequences=True,
                       go_backwards=True)

model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                         input_shape=(10,1)))
 
model.add(Flatten()) 
model.add(Dense(1))

model.compile(optimizer='adam',loss='mae')

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 10, 40)           2760      
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 401       
                                                                 
Total params: 3,161
Trainable params: 3,161
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
7/7 [==============================] - 3s 47ms/step - loss: 193.6072
Epoch 2/10
7/7 [==============================] - 0s 42ms/step - loss: 172.8101
Epoch 3/10
7/7 [==============================] - 0s 33ms/step - loss: 150.4940
Epoch 4/10
7/7 [==============================] - 0s 35ms/step - loss: 123.3466
Epoch 5/10
7/7 [==============================] - 0s 34ms/step - loss: 88.7759
Epoch 6/10
7/7 [==============================] - 0s 30ms/step - loss: 48.5492
Epoch 7/10
7/7 [==============================] - 0s 33ms/step - loss: 10.8256
Epoch 8/10
7/7 [==============================] - 0s 33ms/step - loss: 13.7061
Epoch 9/10
7/7 [==============================] - 0s 28ms/step - loss: 7.4921
Epoch 10/10
7/7 [==============================] - 0s 30ms/step - loss: 5.7194


In [16]:

# EVALUACION
score = model.evaluate(X_test, y_test, batch_size=128)

score

1/1 [==============================] - 0s 495ms/step - loss: 9.4721


9.472057342529297

# Modelo 4
Sin nada (para comparar)

In [17]:
model = Sequential()

model.add(GRU(20, input_shape=(10,1)))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mae')

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                1380      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,401
Trainable params: 1,401
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
7/7 [==============================] - 2s 27ms/step - loss: 230.3847
Epoch 2/10
7/7 [==============================] - 0s 21ms/step - loss: 229.9578
Epoch 3/10
7/7 [==============================] - 0s 20ms/step - loss: 229.5387
Epoch 4/10
7/7 [==============================] - 0s 22ms/step - loss: 229.3765
Epoch 5/10
7/7 [==============================] - 0s 21ms/step - loss: 229.2658
Epoch 6/10
7/7 [==============================] - 0s 20ms/step - loss: 229.1213
Epoch 7/10
7/7 [==============================] - 0s 22ms/step - loss: 228.6493
Epoch 8/10
7/7 [==============================] - 0s 25ms/step - loss: 228.0868
Epoch 9/10
7/7 [==============================] - 0s 20ms/step - loss: 227.9315
Epoch 10/10
7/7 [==============================] - 0s 21ms/step - loss: 227.8376


In [20]:

# EVALUACION
score = model.evaluate(X_test, y_test, batch_size=128)

score

1/1 [==============================] - 0s 356ms/step - loss: 549.6356


549.6355590820312